<a href="https://colab.research.google.com/github/dineshuthayakumar/AIExperiments/blob/main/Chatbot_openai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KF Bot: Simple Chatbot on Kissflow Low Code/No Code Application Development

> Add blockquote



We will scrape the web content as well as our PDFs to answer user queries more effectively.


In [1]:
!pip install --upgrade --quiet  sentence-transformers gradio bs4 pypdf2 chromadb rapidocr-onnxruntime langchain langchain-community langchainhub langchain-openai langchain_huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 72.2 MB/s eta 0:0

## Scraping Kissflow Documentation from the web and using that as the foundation.


In [20]:
# Load, chunk and index the contents of a blog/website.
from langchain_community.document_loaders import WebBaseLoader
import bs4
loader = WebBaseLoader(web_path=(""))

web_urls = [
    "https://community.kissflow.com/t/q6h9ql3/board-overview",
    "https://community.kissflow.com/t/h7h4a18/process-overview",
    "https://community.kissflow.com/t/m1h9qmv/making-a-process-workflow",
    "https://community.kissflow.com/t/83yz7vs/process",
    "https://community.kissflow.com/t/m1hvtsb/process-governance",
    "https://community.kissflow.com/t/m1hvtsb/process-governance-and-auditing",
    "https://community.kissflow.com/category/administering-processes",
    "https://community.kissflow.com/t/p8h9qtp/managing-process-members-and-permissions",
    "https://community.kissflow.com/t/g9h9qtg/process-audit-log",
    "https://community.kissflow.com/t/m1h9qt5/process-settings",
    "https://community.kissflow.com/t/60h9qtj/admin-actions-in-a-process",
    "https://community.kissflow.com/t/h7yz351/connecting-processes",
    "https://community.kissflow.com/t/g9ytv27/connected-processes-overview",
    "https://community.kissflow.com/t/h7yf9xm/associated-flows-for-processes",
    "https://community.kissflow.com/category/process-workflows",
    "https://community.kissflow.com/t/m1h9qmv/making-a-process-workflow",
    "https://community.kissflow.com/t/x2ymwj1/connecting-boards-with-other-boards-and-processes-overview",
    "https://community.kissflow.com/t/m1ymay3/connecting-boards-with-other-boards-and-processes",
    "https://community.kissflow.com/t/h7h4akh/process-connector",
    "https://community.kissflow.com/t/q6yxmsb/process-comments"
]

def load_documents(urls):
    all_docs = []
    for url in urls:
        try:
            loader = WebBaseLoader(web_path=url)
            docs = loader.load()
            all_docs.extend(docs)
        except Exception as e:
            print(f"Failed to load {url}: {e}")
    return all_docs

docs = load_documents(web_urls)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [21]:
docs

[Document(metadata={'source': 'https://community.kissflow.com/t/q6h9ql3/board-overview', 'title': 'Boards overview - Boards - Kissflow Community', 'description': 'With Kissflow Boards, you can create highly adaptable workflows that let you keep track of information and manage work efficiently.', 'language': 'en'}, page_content="\n\n\nBoards overview - Boards - Kissflow Community\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to main content\n\n\n\n\n\n\nCommunity\n\n\n\n\nDeveloper Documentation\nStatus\n\n\nHow to enable Javascript\n\n\n\n\n\n\n\n\n\n\nSign Up\n\n\nLogin\n\n\n\n\n\n\n\n\n\n\nWelcome to the Kissflow Community!\n\n\n\n\n\n\n\n\nSearch for answers here or ask a question\n\n\n\n\n\n\nProduct Documentation\n\n\nðŸŽ¬ Getting started\nðŸ“’ User guide\nðŸ”Ž Quick references\nðŸ’¼ Troubleshooting guide\n\n\n\nProduct Updates\n\n\nPre-release notes\nProduct announcements\nTips and tricks\n\n\nLatest Activity\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nHome\n\n\nProduct Documentation\n\

In [22]:
# Split and manage the texts
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [23]:
# Store embeddings in a Vector DB
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    model_kwargs={"device": "cpu"}
)
vectorstore_web = Chroma.from_documents(documents=splits, embedding=embeddings)
retriever_web = vectorstore_web.as_retriever()

In [24]:
from google.colab import userdata
import os

# If you haven't already installed "openai" in your Colab environment, make sure to:
# !pip install openai

# From LangChain, import the OpenAI LLM
from langchain.chat_models import ChatOpenAI

# Replace this with your actual environment variable (if stored in colab form fields, etc.)
os.environ["OPENAI_API_KEY"] = userdata.get("OpenAPIKey")

# Instantiate the OpenAI LLM (adjust model name if desired, e.g. "gpt-4" or "gpt-3.5-turbo")
llm = ChatOpenAI(model_name="gpt-4")

# Pull in the prompt from your prompt hub
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

# From your LangChain Core imports
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Create your pipeline using OpenAI as the LLM
rag_web = (
    {"context": retriever_web | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
rag_web.invoke("What is the difference between Process and Board?")

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


'In the context of workflows in Kissflow, a board is a platform that can be connected to other boards or processes to automate workflows; a status change in a board can trigger actions in linked boards or processes. On the other hand, a process is a type of workflow that ensures a strict sequential set of steps performed on form data; it involves setting up a form to carry data and making a predefined path for it to follow. The system then automatically routes the requests through these steps until the item is complete.'